In [92]:
from pandas_gbq import read_gbq
import pandas as pd
from tableone import TableOne
from pathlib import Path
import os

In [2]:
PROCEDURES=Path("./procedures")
PROJECT_ID=os.getenv("clinical-entity-extraction") 
# Pass a project-id to a Google Cloud Project.
# Make sure that your google account associated to the google cloud project is the same authorized to access eICU


In [4]:
query=open(PROCEDURES / "transfusion.sql","r").read()
transfusion_df=read_gbq(query,project_id="clinical-entity-extraction")
transfusion_df

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


,patientunitstayid,itemid,offset,item,value,unit
0,168546,13248688,2437,Volume-Transfuse red blood cells,0.00,ml
1,196237,11862542,15305,Volume-Transfuse red blood cells,0.00,ml
2,145079,13352830,19,Volume-Transfuse red blood cells,58.00,ml
3,158233,15141317,8572,Volume-Transfuse red blood cells,68.00,ml
4,188224,11283847,24532,Volume-Transfuse red blood cells,152.00,ml
...,...,...,...,...,...,...
5836,181521,17463171,2809,Volume-Transfuse red blood cells,353.33,ml
5837,152735,12468306,283,Volume-Transfuse red blood cells,339.33,ml
5838,173209,12491750,39027,Volume-Transfuse red blood cells,73.02,ml
5839,230113,10755300,4081,Volume-Transfuse red blood cells,315.58,ml


In [5]:
query=open(PROCEDURES / "hgb.sql","r").read()
hgb_df=read_gbq(query,project_id=PROJECT_ID)
hgb_df

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


,patientunitstayid,itemid,offset,item,value,unit
0,1060530,255063498,-276,Hgb,NaN,g/dL
1,1056817,253922838,-258,Hgb,NaN,g/dL
2,1061601,254477605,-168,Hgb,NaN,g/dL
3,1059992,253657435,-443,Hgb,NaN,g/dL
4,1599400,403769930,23024,Hgb,4.5,g/dL
...,...,...,...,...,...,...
1298703,1118004,270019292,-95,Hgb,16.8,g/dL
1298704,2718048,656498658,-356,Hgb,17.8,g/dL
1298705,1792355,440393649,3660,Hgb,18.8,g/dL
1298706,832289,168547675,-913,Hgb,18.8,g/dL


In [6]:
query=open(PROCEDURES / "blood_loss.sql","r").read()
blood_loss_df=read_gbq(query,project_id=PROJECT_ID)
blood_loss_df

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


,patientunitstayid,itemid,offset,item,value,unit
0,3003577,181995200,1743,Blood Loss,0.0,ml
1,3014604,181527856,2415,Blood Loss,0.0,ml
2,3017558,181004290,1142,Blood Loss,0.0,ml
3,3025991,180799071,1695,Blood Loss,0.0,ml
4,3020118,180893085,1126,Blood Loss,0.0,ml
...,...,...,...,...,...,...
14565,142417,14638203,15602,Blood Loss,1.0,ml
14566,213556,16109849,10714,Blood Loss,225.0,ml
14567,207035,10603344,-6602,Blood Loss,550.0,ml
14568,210543,13898965,9142,Blood Loss,250.0,ml


In [47]:
def filter_ids(df, ids=hgb_df['patientunitstayid']):
    return df[df['patientunitstayid'].isin(ids)]

In [79]:
query=open(PROCEDURES / "apache.sql","r").read()
apache=read_gbq(query,project_id=PROJECT_ID)
apache=apache[apache['patientunitstayid'].isin(ids)]
apache

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


,apacheID,patientunitstayid,apache_score,apache_version,pred_mortality,actual_mortality
0,26570,141168,65,IV,0.079880,EXPIRED
1,26571,141168,65,IVa,0.059099,EXPIRED
2,53135,141194,70,IV,0.098445,ALIVE
3,53136,141194,70,IVa,0.102283,ALIVE
4,8,141203,90,IVa,0.470973,ALIVE
...,...,...,...,...,...,...
297059,3272802,3353251,158,IVa,0.918169,ALIVE
297060,3262173,3353254,35,IV,0.023298,ALIVE
297061,3262174,3353254,35,IVa,0.025801,ALIVE
297062,3246241,3353263,22,IVa,0.012324,ALIVE


In [129]:
query=open(PROCEDURES / "demographics.sql","r").read()
demographics=read_gbq(query,project_id=PROJECT_ID)
demographics=filter_ids(demographics)
demographics['age'] = pd.to_numeric(demographics['age'], errors='coerce')
demographics

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


,patientunitstayid,age,gender,ethnicity,initial_weight,final_weight
0,141168,70.0,Female,Caucasian,84.3,85.8
1,141178,52.0,Female,Caucasian,54.4,54.4
2,141179,52.0,Female,Caucasian,NaN,60.4
3,141194,68.0,Male,Caucasian,73.9,76.7
4,141196,71.0,Male,Caucasian,NaN,63.2
...,...,...,...,...,...,...
200854,3353235,50.0,Male,Caucasian,90.0,99.2
200855,3353237,79.0,Female,Caucasian,78.4,81.4
200856,3353251,73.0,Male,African American,102.0,96.2
200857,3353254,81.0,Male,Caucasian,83.9,92.9


In [130]:
demographics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 189016 entries, 0 to 200858
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   patientunitstayid  189016 non-null  Int64  
 1   age                182314 non-null  float64
 2   gender             189016 non-null  object 
 3   ethnicity          189016 non-null  object 
 4   initial_weight     175951 non-null  float64
 5   final_weight       106368 non-null  float64
dtypes: Int64(1), float64(3), object(2)
memory usage: 10.3+ MB


In [131]:
demog_columns = demographics.columns
demog_categorical=['gender']
demog_nonnormal=['patientunitstayid', 'age']

demog_table = TableOne(demographics,
                       columns=list(demog_columns),
                       categorical=demog_categorical,
                       nonnormal=demog_nonnormal,
                       groupby='ethnicity')

In [132]:
demog_table

Grouped by ethnicity                                                                                                                                                                                                                                                                 
                                                       Missing                         Overall                                                  African American                            Asian                       Caucasian                       Hispanic                Native American                   Other/Unknown
n                                                         1793                            1793                             1793                            20360                             3125                          145914                           7212                           1589                            9023
patientunitstayid, median [Q1,Q3]                            0  1588716.0 [861286.5,2705052.2]  2044244.0 [1315979.0,2203342.0]  1568407.5 [1019778.8,2959051.5]  1858473.0 [1235650.0,2856841.0]  1597921.5 [890582.2,2697990.0]  533723.5 [467672.8,2362459.8]  810032.0 [607305.0,1504683.0]  1741674.0 [879586.5,2720920.5]
age, median [Q1,Q3]                                       6702                64.0 [52.0,75.0]                 65.0 [53.0,75.0]                 58.0 [46.0,68.0]                 65.0 [51.0,76.0]                66.0 [54.0,76.0]               61.0 [47.0,75.0]               53.0 [41.0,65.0]                60.0 [47.0,72.0]
gender, n (%)                                                                         23 (0.0)                         23 (1.3)                                                                                                                                                                                                
                                  Female                                          86821 (45.9)                       716 (39.9)                      9946 (48.9)                      1490 (47.7)                    66718 (45.7)                    3314 (46.0)                     786 (49.5)                     3851 (42.7)
                                  Male                                           102131 (54.0)                      1049 (58.5)                     10409 (51.1)                      1635 (52.3)                    79184 (54.3)                    3896 (54.0)                     803 (50.5)                     5155 (57.1)
                                  Other                                                7 (0.0)                          3 (0.2)                          3 (0.0)                                                                                                                                                        1 (0.0)
                                  Unknown                                             34 (0.0)                          2 (0.1)                          2 (0.0)                                                         12 (0.0)                        2 (0.0)                                                       16 (0.2)
initial_weight, mean (SD)                                13065                     83.9 (27.1)                      84.7 (25.3)                      87.2 (29.9)                      66.5 (17.6)                     84.3 (26.7)                    78.3 (28.2)                    86.4 (29.3)                     80.2 (24.5)
final_weight, mean (SD)                                  82648                     84.3 (34.4)                      85.1 (25.1)                      86.5 (29.6)                      61.3 (19.8)                     84.8 (27.3)                    75.3 (26.5)                   92.8 (207.6)                     80.7 (24.2)